In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, interpolate
import copy
import os 
import pandas as pd
os.chdir('/home/kkotzen/research/PPG_sleepstaging/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser
from src.parsing.utils.comp_ppg_signal_quality import ppg_window_quality, ppg_window_quality_summary, moving_average_filter

def duplicate(x):
    return np.array([[x_]*2 for x_ in x]).flatten()

def hold_line(t, x):
    print(t.shape, x.shape)
    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_

In [19]:
import numpy as np
import copy

from numpy.core._multiarray_umath import ndarray
from scipy import signal, interpolate
import pandas as pd

from scipy.signal import correlate

from src.parsing.armand.feature_comp import bsqi


def f1_score(expected_n_peaks, n_peaks, valid_peaks):
    NB_REF = expected_n_peaks
    NB_TEST = n_peaks
    TP = valid_peaks
    FN = NB_REF - TP
    FP = NB_TEST - TP
    if TP + FN == 0:
        return 0
    Se = TP / (TP + FN)
    if FP + TP == 0:
        return 0
    PPV = TP / (FP + TP)

    if (Se + PPV) > 0:
        return 2 * Se * PPV / (Se + PPV)
    else:
        return 0


def calculate_itervals_forwards(points):
    return np.append((points[1:] - points[0:-1]), np.nan)

def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

# Adapted from Physiozoo moving average filter.
def moving_average_filter(rr, win_samples, percent):
    b_fir = 1 / (2 * win_samples) * np.append(np.ones(win_samples), np.append(0, np.ones(win_samples)))
    rr_moving_average = signal.filtfilt(b_fir, 1, rr)
    rr_filtered = rr.copy()
    rr_filtered[
        ~((rr < (1 + percent / 100) * rr_moving_average) & (rr > (1 - percent / 100) * rr_moving_average))] = np.nan
    nans, x = nan_helper(rr_filtered)
    rr_filtered[nans] = np.interp(x(nans), x(~nans), rr_filtered[~nans])
    return rr_filtered


def find_closest_smaller_value(value, list_of_values):
    for i in reversed(range(len(list_of_values) - 1)):
        if (list_of_values[i] < value):
            return i
    return -1


def find_closest_bigger_value(value, list_of_values):
    for i in range(len(list_of_values) - 1):
        if (list_of_values[i] > value):
            return i
    return -1

def limit_to_high_quality_peaks_range(peaks, quality, quality_window_size=30, peaks_fs=256, smooth_window=10, min_quality=0.9):
    """Returns peaks inside the valid range.
    Smooths quality with a window of 10 and then return peaks from first to last quality meeting"""
    def moving_average(x, w):
        return np.convolve(x, np.ones(w), 'valid') / w

    quality = moving_average(quality, smooth_window)
    start = np.argmax(quality > min_quality)
    end = len(quality) - np.argmax(np.flip(quality)> min_quality)

    start_index = np.argmax(peaks > quality_window_size*peaks_fs*start)
    end_index = np.argmax(peaks > quality_window_size*peaks_fs*end)
    return peaks[start_index:end_index]

def detect_PPG_peak_errors_using_ECG_as_reference(ppg_peaks, ecg_peaks, max_HR_detla=10, plot=False):
    # 0) Make a copy that we will use later.
    ppg_peaks_copy = copy.deepcopy(ppg_peaks)
    start_arg = 0
    end_arg = len(ppg_peaks_copy)

    # 1) Limit the signal ranges to one another.
    if ppg_peaks[-1] > ecg_peaks[-1]:
        end_arg = find_closest_smaller_value(ecg_peaks[-1], ppg_peaks) + 1
    if ppg_peaks[0] < ecg_peaks[0]:
        start_arg = find_closest_bigger_value(ecg_peaks[0], ppg_peaks)

    ppg_peaks = ppg_peaks[start_arg:end_arg]

    # 2) Calculate the RR interval and filter out bad points. Convert to HR estimate
    RR = calculate_itervals_forwards(ecg_peaks) / 256
    RR_filt = moving_average_filter(RR, win_samples=10,
                                    percent=20)  # Moving average window of 10 beats. #Filter @ 20% from moving average
    HR_RR = 60 / (RR_filt)

    # 3) Calculate the PP intervals for the patient. Convert to HR estimate
    PP = calculate_itervals_forwards(ppg_peaks) / 256
    HR_PP = 60 / PP

    # 4) Build the HR band
    band = max_HR_detla
    HR_upper = HR_RR + band
    HR_lower = HR_RR - band
    HR_RR_upper_continous = interpolate.interp1d(ecg_peaks, HR_upper)
    HR_RR_lower_continous = interpolate.interp1d(ecg_peaks, HR_lower)

    # 4) Count where the PP is within the RR band.
    upper = HR_RR_upper_continous(ppg_peaks)
    lower = HR_RR_lower_continous(ppg_peaks)

    valid_peaks = np.zeros(len(ppg_peaks_copy)).astype(bool)
    valid_peaks[start_arg:end_arg] = ((HR_PP < upper) & (HR_PP > lower)) | np.isnan(upper) | np.isnan(lower)

    return valid_peaks

def comp_ppg_signal_quality(ppg_peaks, ecg_peaks, len_ppg, fs=256, window=30, hop=30):
    window_fs = fs * window
    ppg_peak_valid = detect_PPG_peak_errors_using_ECG_as_reference(ppg_peaks, ecg_peaks)

    ppg_features = pd.DataFrame({'PPG-idx': ppg_peaks, 'PPG_valid': ppg_peak_valid})
    ecg_features = pd.DataFrame({'ECG-idx': ecg_peaks})

    windows = np.arange(0, len_ppg, window_fs)

    window_stats = pd.DataFrame()
    for i in (range(windows.shape[0] - 1)):
        window_result = ppg_features[
            (ppg_features['PPG-idx'] >= windows[i]) * (ppg_features['PPG-idx'] < (windows[i + 1]))]
        ecg_result = ecg_features[(ecg_features['ECG-idx'] >= windows[i]) * (ecg_features['ECG-idx'] < windows[i + 1])]

        n_ecg_peaks = ecg_result.shape[0]
        n_ppg_peaks = window_result['PPG_valid'].shape[0]
        valid_peaks = window_result['PPG_valid'].sum()

        window_stats = window_stats.append({'Epoch': i,
                                            'n_peaks': n_ppg_peaks,
                                            'valid_peaks': valid_peaks,
                                            'expected_n_peaks': n_ecg_peaks,
                                            'quality': f1_score(n_ecg_peaks, n_ppg_peaks, valid_peaks)},
                                           ignore_index=True)
    return window_stats

def ppg_window_quality_summary(patient, ppg_detector_quality, ecg_quality, annotator, annotation):
        L = min(len(ecg_quality), len(ppg_detector_quality))
        ppg_detector_quality.loc[0:L, 'ECG_quality'] = ecg_quality[0:L]
        ppg_detector_quality = ppg_detector_quality[0:L]
        ppg_detector_quality = ppg_detector_quality[ppg_detector_quality['ECG_quality'] > 0.85]
        summary = ppg_detector_quality.mean().drop(['Epoch', 'observed_n_peaks', 'n_peaks', 'valid_peaks'])
        df = pd.DataFrame(
            {'patient': patient, 'annotator': annotator, 'annotation': annotation, 'MAE': summary['mean_average_error'],
             'RMSE': summary['rms_error'], 'F1-Old': summary['quality'], 'F1': summary['quality_joachim']}, index=[0])
        return df

def plot_IHR(ppg_peaks, ecg_peaks, fs=256, window=30, max_HR_detla=5):
    # 0) Make a copy that we will use later.
    ppg_peaks_copy = copy.deepcopy(ppg_peaks)
    start_arg = 0
    end_arg = len(ppg_peaks_copy)
        
    # Shift the ECG peaks by 250ms
    print("Shifted")
    ecg_peaks = ecg_peaks + int(0.45*fs)

    # 1) Limit the signal ranges to one another. We have already limited the ECG peaks to high quality only
    if ppg_peaks[-1] > ecg_peaks[-1]:
        end_arg = find_closest_smaller_value(ecg_peaks[-1], ppg_peaks) + 1
    if ppg_peaks[0] < ecg_peaks[0]:
        start_arg = find_closest_bigger_value(ecg_peaks[0], ppg_peaks)
    
    ppg_peaks = ppg_peaks[start_arg:end_arg]

    # 2) Calculate the RR interval and filter out bad points. Convert to HR estimate
    RR = calculate_itervals_forwards(ecg_peaks) / fs
    RR_filt = moving_average_filter(RR, win_samples=10,
                                    percent=50)  # Moving average window of 10 beats. #Filter @ 50% from moving average
    HR_RR = 60 / (RR_filt)

    # 3) Calculate the PP intervals for the patient. Convert to HR estimate
    PP = calculate_itervals_forwards(ppg_peaks) / fs
    HR_PP = 60 / PP
    HR_PP_continous = interpolate.interp1d(ppg_peaks, HR_PP)

    # 4) Build the HR band
    band = max_HR_detla
    HR_upper = HR_RR + band
    HR_lower = HR_RR - band
    HR_RR_upper_continous = interpolate.interp1d(ecg_peaks, HR_upper)
    HR_RR_lower_continous = interpolate.interp1d(ecg_peaks, HR_lower)
    HR_RR_continous = interpolate.interp1d(ecg_peaks, HR_RR)
       
    # 5) Resample all the HR's to 2Hz 
    resample_2Hz = np.arange(ppg_peaks[0], fs/2, ppg_peaks[-1])
    HR_RR = HR_RR_continous(resample_2Hz)
    upper = HR_RR_upper_continous(resample_2Hz)
    lower = HR_RR_lower_continous(resample_2Hz)
    HR_PP = HR_PP_continous(resample_2Hz)
    
    #6) Count the false positives and negatives
    peak_classification = np.zeros(len(resample_2Hz)).astype(bool)
    peak_classification[start_arg:end_arg] = ((HR_PP < upper) & (HR_PP > lower)) | np.isnan(upper) | np.isnan(lower)

    false_positives = np.zeros(len(resample_2Hz)).astype(bool)
    false_positives[start_arg:end_arg] = (HR_PP > upper) & (~np.isnan(upper))
    false_negatives = np.zeros(len(resample_2Hz)).astype(bool)
    false_negatives[start_arg:end_arg] = (HR_PP < lower) & (~np.isnan(lower))

    ihr = np.empty(len(ppg_peaks_copy))
    ihr[:] = np.nan
    ihr[start_arg:end_arg] = HR_RR_continous(ppg_peaks)

    ipr = np.empty(len(ppg_peaks_copy))
    ipr[:] = np.nan
    ipr[start_arg:end_arg] = HR_PP

#     ipr[~peak_classification] = np.nan

    plt.close("all")
    fig = plt.figure()
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
    ax3 = fig.add_subplot(gs[:, 1], sharex=ax1)

#     ax1.plot(upper, label="Upper IHR")
#     ax1.plot(lower,  label="Lower IHR")
    ax1.plot(HR_PP,  label="IPR")
    ax1.plot(HR_RR, label='IHR')
    ax1.legend(loc='upper right')
    ax1.set_ylabel("Rate [BPM]")
  
    ax2.plot(ihr - ipr)
    ax2.set_ylabel("IHR-IPR Error [BPM]")
    ax2.set_xlabel("Time (N/A)")
   
    ax3.plot(upper, label="Upper Bound IHR")
    ax3.plot(lower,  label="Lower Bound IHR")
    ax3.plot(HR_PP,  'x', label="IPR at given peak")
    ax3.set_ylabel("Rate [BPM]")
    ax3.yaxis.tick_right()
    ax3.yaxis.set_label_position("right")
    ax3.set_xlabel("Time (N/A)")
    ax3.set_xticks([])
    ax3.legend(loc='upper right')
    print(np.nanmean(np.abs(ihr- ipr)))
    
    plt.tight_layout()

In [20]:
dl = MESAParser()
patients = ['6632'] #dl.database_all_patient_IDs

ppg = dl.load_signal(patients[0], 'Pleth')
ecg = dl.load_signal(patients[0], 'EKG')

ecg_peaks = dl.load_annotation(patients[0], 'EKG', 'epltd0', 'Peaks').astype(int)
valid_ecg_peaks = dl.load_annotation(patients[0], 'EKG', 'rpoint', 'Peaks').astype(int)
ecg_peaks = ecg_peaks[ecg_peaks < valid_ecg_peaks[-1]]
ecg_peaks = ecg_peaks
ecg_quality = dl.load_quality(patients[0], 'EKG')

#Load the peaks
Aboy_ppg_peaks = dl.load_annotation(patients[0], 'Pleth', 'Aboy', 'Peaks')

#Limit them to the reference ECG
Aboy_ppg_peaks = Aboy_ppg_peaks[Aboy_ppg_peaks<ecg_peaks[-1]]

plot_IHR(Aboy_ppg_peaks, ecg_peaks)


Shifted


ValueError: could not broadcast input array from shape (0) into shape (37708)

In [ ]:
valid, invalid, doubles, missed_ecg = classify_peaks_and_ecg(Aboy_ppg_peaks, delayed_ecg_peaks)

plt.close("all")
plt.figure(figsize=(10,5))
plt.plot(ppg)
plt.plot(valid, ppg[valid], 'o', color='green', label='True Peaks')
plt.plot(invalid, ppg[invalid], 'o', color='red', label="False Peaks")
plt.plot(missed_ecg, ppg[missed_ecg], 'o', color='orange', label="Missed ECG Peaks")
plt.plot(doubles, ppg[doubles], 'x', color='black', label="Double Peaks")
plt.plot(delayed_ecg_peaks, ppg[delayed_ecg_peaks], 'x', label='ECG Forecast')
plt.ylabel('PPG Amplitude',fontsize=14)
plt.xlabel("Time (Samples)", fontsize=14)
plt.legend(loc='upper right')